In [31]:
import json
import ast
import re
from textblob import TextBlob
import pandas as pd
import numpy as np
from typing import Optional
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

Rutas de los datasets

In [32]:
ruta_steam_games = 'datasets/output_steam_games.json'
ruta_users_items = 'datasets/australian_users_items.json'
ruta_users_reviews = 'datasets/australian_user_reviews.json'

ruta_parquet_games = "datasets/data_steam_games.parquet"
ruta_parquet_reviews = "datasets/data_reviews.parquet"
ruta_parquet_items = "datasets/data_items.parquet"

ruta_games_csv = 'datasets/steam_games.csv'
ruta_items_csv = 'datasets/users_items.csv'
ruta_reviews_csv = 'users_reviews.csv'

importación de los datasets

In [33]:
df_games = pd.read_parquet(ruta_parquet_games)
df_items = pd.read_parquet(ruta_parquet_items)
df_reviews = pd.read_parquet(ruta_parquet_reviews)

Función **developer**`( desarrollador : str )`: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 
- Ejemplo de retorno:

| Año | Cantidad de Items | Contenido Free |
| --- | --- | --- |
| 2023 | 50 | 27% |
| 2022 | 45 | 25% |
| xxxx | xx | xx% |

In [46]:
def developer(desarrollador: str):
    # Filtrar los juegos del desarrollador especificado
    juegos_del_desarrollador = df_games[df_games['developer'] == desarrollador]

    # Combinar los DataFrames para obtener los juegos de ese desarrollador
    combined_df = pd.merge(df_items, juegos_del_desarrollador, on="item_id", how="inner")

    # Agrupar por año y contar la cantidad de elementos
    cantidad_por_año = combined_df.groupby('year')['item_id'].count().reset_index()
    cantidad_por_año.columns = ['Año', 'Cantidad de Items']

    # Calcular el porcentaje de contenido gratuito (Free) por año
    contenido_free_por_año = combined_df[combined_df['price'] == "Free To Play"].groupby('year')['item_id'].count() / cantidad_por_año['Cantidad de Items'] * 100
    contenido_free_por_año = contenido_free_por_año.reset_index()
    contenido_free_por_año.columns = ['Año', 'Contenido Free']

    # Combinar los DataFrames de cantidad y contenido gratuito
    resultado = pd.merge(cantidad_por_año, contenido_free_por_año, on="Año", how="left")

    return resultado


Prueba de la función

In [47]:
desarrollador = "Studio Wildcard"
resultados = developer(desarrollador)
resultados

,Cantidad de Items,Año,Contenido Free


Función **userdata**( User_id : str ) - Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Función **UserForGenre**`( genero : str )`: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [36]:
def UserForGenre(genero: str):
    # Filtrar df_games para obtener solo las filas que contienen el género especificado
    filtered_games = df_games[df_games['genres'].str.contains(genero, case=False, na=False)]

    # Filtrar df_items para reducir el conjunto de datos a las columnas necesarias
    df_items_filtered = df_items[['user_id', 'item_id', 'playtime_forever']]

    # Combinar los DataFrames filtrados en uno solo usando "item_id" como clave
    combined_df = pd.merge(df_items_filtered, filtered_games, on="item_id", how="inner")

    # Agrupar por usuario y año, sumar las horas jugadas y encontrar el usuario con más horas jugadas
    result_df = combined_df.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()
    max_user = result_df.loc[result_df['playtime_forever'].idxmax()]

    # Convertir las horas jugadas de minutos a horas en el DataFrame resultante
    result_df['playtime_forever'] = result_df['playtime_forever'] / 60
    result_df['playtime_forever'] = result_df['playtime_forever'].round(0)

    # Crear una lista de acumulación de horas jugadas por año
    accumulation = result_df.groupby('year')['playtime_forever'].sum().reset_index()
    accumulation = accumulation.rename(columns={'year': 'Año', 'playtime_forever': 'Horas'})
    accumulation_list = accumulation.to_dict(orient='records')

    return {"Usuario con más horas jugadas para el género " + genero: max_user['user_id'], "Horas jugadas": accumulation_list}


Prueba de la función

In [40]:
genero = "Strategy"
usuario_por_genero = UserForGenre(genero)
usuario_por_genero

{'Usuario con más horas jugadas para el género Strategy': 'shinomegami',
 'Horas jugadas': [{'Año': 1988, 'Horas': 221.0},
  {'Año': 1990, 'Horas': 0.0},
  {'Año': 1991, 'Horas': 0.0},
  {'Año': 1993, 'Horas': 3010.0},
  {'Año': 1994, 'Horas': 18.0},
  {'Año': 1995, 'Horas': 899.0},
  {'Año': 1996, 'Horas': 140.0},
  {'Año': 1997, 'Horas': 2204.0},
  {'Año': 1998, 'Horas': 419.0},
  {'Año': 1999, 'Horas': 9527.0},
  {'Año': 2000, 'Horas': 506.0},
  {'Año': 2001, 'Horas': 15595.0},
  {'Año': 2002, 'Horas': 8019.0},
  {'Año': 2003, 'Horas': 122142.0},
  {'Año': 2004, 'Horas': 61368.0},
  {'Año': 2005, 'Horas': 16763.0},
  {'Año': 2006, 'Horas': 286645.0},
  {'Año': 2007, 'Horas': 52547.0},
  {'Año': 2008, 'Horas': 135119.0},
  {'Año': 2009, 'Horas': 529311.0},
  {'Año': 2010, 'Horas': 2347056.0},
  {'Año': 2011, 'Horas': 574957.0},
  {'Año': 2012, 'Horas': 1426094.0},
  {'Año': 2013, 'Horas': 2420832.0},
  {'Año': 2014, 'Horas': 838494.0},
  {'Año': 2015, 'Horas': 1369682.0},
  {'Año': 2

Función **best_developer_year**`( año : int )`- Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [38]:
def best_developer_year(año: int):
    # Filtra los juegos del año especificado en df_games
    juegos_del_año = df_games[df_games['year'] == año]

    # Combinación de DataFrames para obtener los juegos recomendados en ese año
    combined_df = pd.merge(juegos_del_año, df_reviews, on="item_id", how="inner")

    # Filtra los juegos recomendados con comentarios positivos
    juegos_recomendados = combined_df[(combined_df['recommend'] == True) & (combined_df['sentiment_analysis'] == 2)]

    # Agrupa por desarrollador y cuenta las recomendaciones
    desarrolladores_recomendados = juegos_recomendados['developer'].value_counts().reset_index()
    desarrolladores_recomendados.columns = ['developer', 'recommend_count']

    # Ordena en orden descendente y toma los 3 principales
    top_desarrolladores = desarrolladores_recomendados.nlargest(3, 'recommend_count')

    # Formatea el resultado en un formato de lista de diccionarios
    resultado = [{"Puesto {}: {}".format(i + 1, row['developer']): row['recommend_count']} for i, row in top_desarrolladores.iterrows()]

    return resultado



prueba de la función

In [48]:
año = 2017
top_desarrolladores = best_developer_year(año)
top_desarrolladores

[{'Puesto 1: Smartly Dressed Games': 522},
 {'Puesto 2: Freejam': 171},
 {'Puesto 3: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC': 145}]

Función **developer**`( desarrolladora : str )` - Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

- Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [54]:
def developer(desarrolladora: str):
    # Filtrar las reseñas por el desarrollador dado
    reseñas_desarrolladora = df_reviews[df_reviews['user_id'].isin(df_items[df_items['item_id'].isin(df_games[df_games['developer'] == desarrolladora]['item_id'])]['user_id'])]

    # Contar las reseñas con sentimiento positivo y negativo
    sentimiento_positivo = reseñas_desarrolladora[reseñas_desarrolladora['sentiment_analysis'] == 2].shape[0]
    sentimiento_negativo = reseñas_desarrolladora[reseñas_desarrolladora['sentiment_analysis'] == 0].shape[0]

    # Crear el diccionario de resultados
    resultado = {desarrolladora: {'Positive': sentimiento_positivo, 'Negative': sentimiento_negativo}}

    return resultado

Prueba de la función

In [55]:
desarrolladora = "Valve"
desarrollador = developer(desarrolladora)
desarrollador

{'Valve': {'Positive': 27944, 'Negative': 10758}}

**Funciónes ML**

*Si es un sistema de recomendación item-item:*

Función **recomendacion_juego**`( id de producto )` -  Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

*Si es un sistema de recomendación user-item:*

Función **recomendacion_usuario**`( id de usuario )` - Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.